In [1]:
source("/home/user/data2/lit/bin/lit_utils.R")
source("/home/user/data3/lit/project/sORFs/sORFs.utils.R")
lib_text()
lib_plot()

In [6]:
all_sample_sep_m_meta_path="../../results/1/S4/all_sample_sep_m_meta.txt"
output_path <- "../../results/1/S5"
create_path(output_path)
orfs_path <- "../../results/1/orfs_merged_final.tsv"
fread_c(orfs_path) -> orfs

In [7]:
merge_replicate <- function(df){
    # 合并重复
    df$Sample_merge_replicate <- gsub("21pcw(_1|_2|_3)", "21pcw", df$Sample)
    df %>% mutate(Sample=NULL) %>% rename(Sample=Sample_merge_replicate) -> df_1
    return(df_1)
}

fread_c(all_sample_sep_m_meta_path) -> all_sample_sep_m_meta

df <- merge_replicate(all_sample_sep_m_meta)
protein_stats_v1 <- df %>%
  group_by(Protein) %>%
  summarise(
    Unique_Methods = n_distinct(Enrichment),  # 被多少种不同富集方法鉴定到
    Unique_Enzymes = n_distinct(Eyzyme),  # 被多少种不同酶切方法鉴定到
    Total_Observations = n(),                 # 被鉴定到的总次数
    Supported_Enrichments = paste(sort(unique(Enrichment)), collapse = ", "),  # 支持的富集方法（去重后逗号分隔）
    Supported_Enzymes = paste(sort(unique(Eyzyme)), collapse = ", ")  # 支持的酶切方法（去重后逗号分隔）
  )
  
fwrite_c(protein_stats_v1[,"Protein"],o("protein.id.txt"))
  
  # 得到蛋白质对应的序列
# system("bash S5.bio.sh")
# read.table(o("protein.id.tab")) -> protein_id_seq
orfs[,c("ORF_id","ORF_seq")]  -> protein_id_seq
colnames(protein_id_seq) <- c("Protein","Seq")
merge(protein_stats_v1,protein_id_seq,by="Protein") -> protein_stats_v1_a_seq

protein_stats_v1_a_seq %>% filter(Unique_Methods==1|Unique_Methods==4) -> unique_methods_protein
unique_methods_protein[,c("Protein","Seq","Supported_Enrichments")] -> unique_methods_protein_df
table(unique_methods_protein_df$Supported_Enrichments)
fwrite_c(unique_methods_protein_df,o("unique_methods_protein.txt"))

protein_stats_v1_a_seq %>% filter(Unique_Enzymes==1) -> unique_enzymes_protein
unique_enzymes_protein[,c("Protein","Seq","Supported_Enzymes")] -> unique_enzymes_protein_df
fwrite_c(unique_enzymes_protein_df,o("unique_enzymes_protein.txt"))


                 C8 C8, MWCO, PAGE, PCP                MWCO                PAGE 
              10387                1186                4916                2999 
                PCP 
              17249 